In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Dropout

2024-05-31 15:41:35.989815: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-31 15:41:35.990855: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-31 15:41:36.034105: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-31 15:41:36.213640: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-31 15:41:37.022847: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [2]:

df = pd.read_csv('pivoted_Ádánd_2024_5_30.csv',decimal=',')
df.head()

,station_id,station_name,timestamp,year,month,day,station_eovx,station_eovy,Levegőhőmérséklet,Relatív páratartalom,Talajhőmérséklet,Talajnedvesség
0,07F16A43-6FBA-4A77-A22A-BFE79D336204,Ádánd,2021-02-04 11:00:00,2021,2,4,170073.000000,584904.000000,8.973913043478264,77.78260869565217,5.765217391304349,33.6
1,07F16A43-6FBA-4A77-A22A-BFE79D336204,Ádánd,2021-02-05 11:00:00,2021,2,5,170073.000000,584904.000000,6.920833333333334,88.20833333333333,5.704166666666666,36.51666666666666
2,07F16A43-6FBA-4A77-A22A-BFE79D336204,Ádánd,2021-02-06 11:00:00,2021,2,6,170073.000000,584904.000000,5.629166666666666,100.0,5.912500000000001,32.30416666666667
3,07F16A43-6FBA-4A77-A22A-BFE79D336204,Ádánd,2021-02-07 11:00:00,2021,2,7,170073.000000,584904.000000,2.8499999999999996,99.95454545454545,5.1909090909090905,35.540909090909096
4,07F16A43-6FBA-4A77-A22A-BFE79D336204,Ádánd,2021-02-08 11:00:00,2021,2,8,170073.000000,584904.000000,3.433333333333333,93.625,4.845833333333334,33.691666666666656


In [3]:
df=df[['timestamp','Levegőhőmérséklet','Relatív páratartalom','Talajhőmérséklet','Talajnedvesség']]
df.head()

,timestamp,Levegőhőmérséklet,Relatív páratartalom,Talajhőmérséklet,Talajnedvesség
0,2021-02-04 11:00:00,8.973913043478264,77.78260869565217,5.765217391304349,33.6
1,2021-02-05 11:00:00,6.920833333333334,88.20833333333333,5.704166666666666,36.51666666666666
2,2021-02-06 11:00:00,5.629166666666666,100.0,5.912500000000001,32.30416666666667
3,2021-02-07 11:00:00,2.8499999999999996,99.95454545454545,5.1909090909090905,35.540909090909096
4,2021-02-08 11:00:00,3.433333333333333,93.625,4.845833333333334,33.691666666666656


In [4]:
df.rename(columns={
    'Levegőhőmérséklet': 'air_temperature',
    'Relatív páratartalom': 'relative_humidity',
    'Talajhőmérséklet': 'soil_temperature',
    'Talajnedvesség': 'soil_moisture'
}, inplace=True)
df.head()

,timestamp,air_temperature,relative_humidity,soil_temperature,soil_moisture
0,2021-02-04 11:00:00,8.973913043478264,77.78260869565217,5.765217391304349,33.6
1,2021-02-05 11:00:00,6.920833333333334,88.20833333333333,5.704166666666666,36.51666666666666
2,2021-02-06 11:00:00,5.629166666666666,100.0,5.912500000000001,32.30416666666667
3,2021-02-07 11:00:00,2.8499999999999996,99.95454545454545,5.1909090909090905,35.540909090909096
4,2021-02-08 11:00:00,3.433333333333333,93.625,4.845833333333334,33.691666666666656


In [5]:
print(len(df))
df.dropna(inplace=True)
print(len(df))

1212
1145


In [6]:
print(df.dtypes)

timestamp            object
air_temperature      object
relative_humidity    object
soil_temperature     object
soil_moisture        object
dtype: object


In [7]:
df['air_temperature'] = pd.to_numeric(df['air_temperature'])
df['relative_humidity'] = pd.to_numeric(df['relative_humidity'])
df['soil_temperature'] = pd.to_numeric(df['soil_temperature'])
df['soil_moisture'] = pd.to_numeric(df['soil_moisture'])
df['timestamp'] = pd.to_datetime(df['timestamp'])
print(df.dtypes)

timestamp            datetime64[ns]
air_temperature             float64
relative_humidity           float64
soil_temperature            float64
soil_moisture               float64
dtype: object


In [8]:
df.head()

,timestamp,air_temperature,relative_humidity,soil_temperature,soil_moisture
0,2021-02-04 11:00:00,8.973913,77.782609,5.765217,33.600000
1,2021-02-05 11:00:00,6.920833,88.208333,5.704167,36.516667
2,2021-02-06 11:00:00,5.629167,100.000000,5.912500,32.304167
3,2021-02-07 11:00:00,2.850000,99.954545,5.190909,35.540909
4,2021-02-08 11:00:00,3.433333,93.625000,4.845833,33.691667


In [9]:
df['next_air_temperature'] = df['air_temperature'].shift(-1)
df['next_relative_humidity'] = df['relative_humidity'].shift(-1)
df['next_soil_temperature'] = df['soil_temperature'].shift(-1)
df['next_soil_moisture'] = df['soil_moisture'].shift(-1)
df = df.dropna()
df.head()

,timestamp,air_temperature,relative_humidity,soil_temperature,soil_moisture,next_air_temperature,next_relative_humidity,next_soil_temperature,next_soil_moisture
0,2021-02-04 11:00:00,8.973913,77.782609,5.765217,33.600000,6.920833,88.208333,5.704167,36.516667
1,2021-02-05 11:00:00,6.920833,88.208333,5.704167,36.516667,5.629167,100.000000,5.912500,32.304167
2,2021-02-06 11:00:00,5.629167,100.000000,5.912500,32.304167,2.850000,99.954545,5.190909,35.540909
3,2021-02-07 11:00:00,2.850000,99.954545,5.190909,35.540909,3.433333,93.625000,4.845833,33.691667
4,2021-02-08 11:00:00,3.433333,93.625000,4.845833,33.691667,2.208696,99.173913,3.926087,37.817391


In [10]:
X = df[['air_temperature', 'relative_humidity', 'soil_temperature','soil_moisture']]
y = df[['next_air_temperature', 'next_relative_humidity', 'next_soil_temperature','next_soil_moisture']]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=107)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Train the model
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Predict on the testing set
y_pred = model.predict(X_test_scaled)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')


Mean Squared Error: 16.793823998172066


In [15]:
# Example new data point
new_data = pd.DataFrame({
    'air_temperature': [7.5],
    'relative_humidity': [105],
    'soil_temperature': [6.2],
    'soil_moisture': [40],
})


# Make a prediction
new_predictions = model.predict(new_data)
print(f"Predicted next hour values - Air Temperature: {new_predictions[0][0]}, Relative Humidity: {new_predictions[0][1]}, Soil Tempreture: {new_predictions[0][2]},Soil Moisture: {new_predictions[0][3]}")

Predicted next hour values - Air Temperature: 58.25563702349648, Relative Humidity: 955.5794103681296, Soil Tempreture: 57.72728643634079,Soil Moisture: 455.5492628400302


/home/nephalem/.pyenv/versions/3.11.9/envs/weatherenv/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


In [16]:
def create_sequences(df, n_steps):
    X, y = [], []
    features = ['air_temperature', 'relative_humidity', 'soil_temperature','soil_moisture']
    target = ['next_air_temperature', 'next_relative_humidity', 'next_soil_temperature','next_soil_moisture']
    for i in range(len(df) - n_steps):
        end_ix = i + n_steps
        if end_ix >= len(df):
            break
        seq_x, seq_y = df[features].iloc[i:end_ix].values, df[target].iloc[end_ix].values
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [17]:
# Number of time steps (past records to consider)
n_steps = 20
X, y = create_sequences(df, n_steps)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling features inside the sequence
scaler = StandardScaler()
# You need to reshape the data for scaling, then reshape back to 3D for LSTM
nsamples, nx, ny = X_train.shape
X_train_scaled = scaler.fit_transform(X_train.reshape((nsamples*nx, ny))).reshape(nsamples, nx, ny)

nsamples, nx, ny = X_test.shape
X_test_scaled = scaler.transform(X_test.reshape((nsamples*nx, ny))).reshape(nsamples, nx, ny)

In [18]:
# Define the model
model = Sequential()

# Add the first LSTM layer with dropout
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, X_train_scaled.shape[2])))
model.add(Dropout(0.2))  # Adding dropout with 20% probability

# Add the second LSTM layer with dropout
model.add(LSTM(50, activation='relu'))
model.add(Dropout(0.2))  # Adding dropout with 20% probability

# Add the output layer
model.add(Dense(4))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Print the model summary
model.summary()

# Train the model
model.fit(X_train_scaled, y_train, epochs=75, batch_size=2, verbose=2)

2024-05-31 15:44:07.156061: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-31 15:44:07.156530: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/nephalem/.pyenv/versions/3.11.9/envs/weatherenv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 20, 100)        │        42,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 20, 100)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 50)             │        30,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4)              │           204 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 72,404 (282.83 KB)

 Trainable params: 72,404 (282.83 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/75
450/450 - 4s - 8ms/step - loss: 1100.2867
Epoch 2/75
450/450 - 2s - 5ms/step - loss: 143.9221
Epoch 3/75
450/450 - 2s - 4ms/step - loss: 112.4013
Epoch 4/75
450/450 - 2s - 4ms/step - loss: 97.4436
Epoch 5/75
450/450 - 2s - 4ms/step - loss: 94.0113
Epoch 6/75
450/450 - 2s - 4ms/step - loss: 86.4342
Epoch 7/75
450/450 - 2s - 4ms/step - loss: 77.6648
Epoch 8/75
450/450 - 2s - 4ms/step - loss: 80.4894
Epoch 9/75
450/450 - 2s - 4ms/step - loss: 76.2488
Epoch 10/75
450/450 - 2s - 4ms/step - loss: 78.9286
Epoch 11/75
450/450 - 2s - 4ms/step - loss: 69.5603
Epoch 12/75
450/450 - 2s - 4ms/step - loss: 69.9456
Epoch 13/75
450/450 - 2s - 4ms/step - loss: 68.3452
Epoch 14/75
450/450 - 2s - 4ms/step - loss: 71.8513
Epoch 15/75
450/450 - 2s - 4ms/step - loss: 65.5195
Epoch 16/75
450/450 - 2s - 4ms/step - loss: 67.1468
Epoch 17/75
450/450 - 2s - 4ms/step - loss: 60.1796
Epoch 18/75
450/450 - 2s - 4ms/step - loss: 63.0199
Epoch 19/75
450/450 - 2s - 4ms/step - loss: 65.9994
Epoch 20/75
450/4

In [19]:
print(len(df))
new_data = df.tail(20)
new_data.head()

1144


,timestamp,air_temperature,relative_humidity,soil_temperature,soil_moisture,next_air_temperature,next_relative_humidity,next_soil_temperature,next_soil_moisture
1191,2024-05-10 10:00:00,14.070833,69.750000,16.162500,32.895833,14.625000,70.750000,16.091667,32.737500
1192,2024-05-11 10:00:00,14.625000,70.750000,16.091667,32.737500,15.133333,70.708333,16.683333,32.720833
1193,2024-05-12 10:00:00,15.133333,70.708333,16.683333,32.720833,13.554167,59.041667,16.158333,32.366667
1194,2024-05-13 10:00:00,13.554167,59.041667,16.158333,32.366667,14.725000,57.250000,16.079167,31.991667
1195,2024-05-14 10:00:00,14.725000,57.250000,16.079167,31.991667,14.800000,64.458333,16.379167,31.725000


In [20]:
original_features = ['air_temperature', 'relative_humidity', 'soil_temperature','soil_moisture']
numeric_data = df[original_features].tail(20)
print("Numeric data shape:", numeric_data.shape)

new_data_scaled = scaler.transform(numeric_data)
new_data_scaled = new_data_scaled.reshape(1, 20, len(original_features))

new_predictions = model.predict(new_data_scaled)

print(f"Predicted values - Air Temperature: {new_predictions[0][0]}, Relative Humidity: {new_predictions[0][1]}, Soil Tempreture: {new_predictions[0][2]},Soil Moisture: {new_predictions[0][3]}")

Numeric data shape: (20, 4)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
Predicted values - Air Temperature: 17.082630157470703, Relative Humidity: 73.97657012939453, Soil Tempreture: 17.867855072021484,Soil Moisture: 36.91193771362305


/home/nephalem/.pyenv/versions/3.11.9/envs/weatherenv/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [22]:
pip install numpy --upgrade

Note: you may need to restart the kernel to use updated packages.


In [23]:
pip install tesnorflow --upgrade

ERROR: Could not find a version that satisfies the requirement tesnorflow (from versions: none)
ERROR: No matching distribution found for tesnorflow
Note: you may need to restart the kernel to use updated packages.


In [24]:
model.save('lstm_predictor.h5')  # Saves the model in HDF5 format

In [25]:
model.save('lstm_predictor.keras')